In [3]:
from calculations.damage_calculator import damage_calc, raise_for_invalid_inputs
from calculations.dataclasses_and_enums import OneTalents, SevenTalents, SixteenTalents

In [4]:
time_series, damage_series = damage_calc(7, 10, num_clones=0, num_clones_attacking=0, one_talent=OneTalents.WAYOFTHEBLADE, seven_talent=SevenTalents.BURNINGBLADE, sixteen_talent=SixteenTalents.NONE)

268.4500829641114 CRIT samuro
546.967044039377 CRIT -W samuro
694.6145896696382 AA samuro
842.2621352998995 AA samuro
1130.8459744863192 CRIT samuro
1285.2047721906833 AA samuro
1439.5635698950473 AA samuro
1738.2142871926212 CRIT samuro
1892.5730848969852 AA samuro
2046.9318826013491 AA samuro
2345.582599898923 CRIT samuro
2499.941397603287 AA samuro
2654.300195307651 AA samuro
2952.950912605225 CRIT samuro
3107.309710309589 AA samuro
3261.6685080139528 AA samuro
3560.3192253115267 CRIT samuro
3714.6780230158906 AA samuro


In [5]:
damage_series

[268.4500829641114,
 546.967044039377,
 694.6145896696382,
 842.2621352998995,
 1130.8459744863192,
 1285.2047721906833,
 1439.5635698950473,
 1738.2142871926212,
 1892.5730848969852,
 2046.9318826013491,
 2345.582599898923,
 2499.941397603287,
 2654.300195307651,
 2952.950912605225,
 3107.309710309589,
 3261.6685080139528,
 3560.3192253115267,
 3714.6780230158906]

In [6]:
time_series

[0,
 0.1875,
 0.7863023952095809,
 1.3851047904191618,
 1.9839071856287427,
 2.5827095808383236,
 3.1815119760479043,
 3.780314371257485,
 4.379116766467066,
 4.977919161676646,
 5.576721556886227,
 6.175523952095808,
 6.774326347305388,
 7.373128742514969,
 7.97193113772455,
 8.570733532934131,
 9.169535928143713,
 9.768338323353294]

In [5]:
import matplotlib.pyplot as plt